### purpose 

calculate the fitness of populations in the climate outlier scenarios for all 2250 seeds

In [1]:
from pythonimports import *
from myfigs import histo_box

import MVP_00_start_pipeline as mvp00
import MVP_10_train_lfmm2_offset as mvp10
import MVP_summary_functions as mvp

envdir = makedir('/home/b.lind/offsets/climate_outlier_runs/garden_files')
slimdir = '/work/lotterhos/MVP-NonClinalAF/sim_output_20220428'
fitness_dir = makedir('/home/b.lind/offsets/climate_outlier_runs/fitness_mats')

lview, dview = get_client(cluster_id='1677769331-fkj5', profile='lotterhos')

mvp.latest_commit()
session_info.show()

36 36
#########################################################
Today:	March 02, 2023 - 16:31:06
python version: 3.8.5

Current commit of pythonimports:
commit f7e63eead1814fe6ce13da1f593ebfb19efdc4c2  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Wed Feb 15 14:14:00 2023 -0500

Current commit of MVP_offsets:
commit d24363688641c3c7e26b5202d74aefc5db4de83c  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Wed Mar 1 16:34:25 2023 -0500
#########################################################



In [2]:
nones = [os.remove(f) for f in pbar(fs(fitness_dir, endswith='.txt'))]  # redefined variables

100%|███████████████| 18000/18000 [00:52<00:00, 345.18it/s]


# create climate data

In [3]:
def create_new_envfiles(outerdir, garden_dir):
    print(ColorText('\nCreating climate outlier environmental files').bold().custom('gold'))
    # read in arbitrary env file for correct formatting
    arbitrary_file = fs(op.join(outerdir, 'gradient_forests/fitting/garden_files'), pattern='_envfile_GFready_pooled_1.txt')[0]
    blank_env = pd.read_table(arbitrary_file, index_col=0)

    for col in blank_env.columns:
        assert luni(blank_env[col]) == 1

    new_envfiles = []
    for newval in pbar(mvp.new_envs):
        df = blank_env.copy()
        df['sal_opt'] = -newval  # new envs are in top left of landscape (negative sal_opt, positive temp_opt)
        df['temp_opt'] = newval

        file = op.join(garden_dir, f'envfile_GF_ready_pooled_envval_{newval}.txt')

        df.to_csv(file, sep='\t', index=True)

#         print(file)

        new_envfiles.append(file)

    return new_envfiles



In [4]:
envfiles = create_new_envfiles('/home/b.lind/offsets/run_20220919_0-225', envdir)

envfiles


Creating climate outlier environmental files


100%|███████████████| 12/12 [00:00<00:00, 190.59it/s]


['/home/b.lind/offsets/climate_outlier_runs/garden_files/envfile_GF_ready_pooled_envval_0.txt',
 '/home/b.lind/offsets/climate_outlier_runs/garden_files/envfile_GF_ready_pooled_envval_1.1.txt',
 '/home/b.lind/offsets/climate_outlier_runs/garden_files/envfile_GF_ready_pooled_envval_1.5.txt',
 '/home/b.lind/offsets/climate_outlier_runs/garden_files/envfile_GF_ready_pooled_envval_1.75.txt',
 '/home/b.lind/offsets/climate_outlier_runs/garden_files/envfile_GF_ready_pooled_envval_2.0.txt',
 '/home/b.lind/offsets/climate_outlier_runs/garden_files/envfile_GF_ready_pooled_envval_2.25.txt',
 '/home/b.lind/offsets/climate_outlier_runs/garden_files/envfile_GF_ready_pooled_envval_2.5.txt',
 '/home/b.lind/offsets/climate_outlier_runs/garden_files/envfile_GF_ready_pooled_envval_2.75.txt',
 '/home/b.lind/offsets/climate_outlier_runs/garden_files/envfile_GF_ready_pooled_envval_3.0.txt',
 '/home/b.lind/offsets/climate_outlier_runs/garden_files/envfile_GF_ready_pooled_envval_3.25.txt',
 '/home/b.lind/off

In [5]:
envdata = {}
for f in envfiles:
    val = float(op.basename(f).split("_")[-1].split('.txt')[0])
    
    envdata[val] = -val, val
    
    print(envdata[val])

(-0.0, 0.0)
(-1.1, 1.1)
(-1.5, 1.5)
(-1.75, 1.75)
(-2.0, 2.0)
(-2.25, 2.25)
(-2.5, 2.5)
(-2.75, 2.75)
(-3.0, 3.0)
(-3.25, 3.25)
(-3.5, 3.5)
(-3.75, 3.75)


# get a list of seeds

In [6]:
seeds = mvp00.get_seeds(slimdir)

len(seeds)

2250

# calc fitness to each scenario

In [7]:
def calc_R_fitness(*args):
    import subprocess
    
    args = [str(arg) for arg in args]
    
    Rscript = '/home/b.lind/anaconda3/envs/MVP_env_R4.0.3/bin/Rscript'
#     script_file = '/home/b.lind/delete/fits_calc_ind.R'
#     script_file = '/home/b.lind/delete/fits_calc_ind_all.R'
    script_file = '/home/b.lind/code/MVP-offsets/01_src/MVP_climate_outlier_fitness_calculator.R'
    
    output = subprocess.check_output(
        [
            Rscript,
            script_file,
            *args
        ]
    )

    pass

In [8]:
params = mvp10.read_params_file('/work/lotterhos/MVP-NonClinalAF/src')

params.head(2)

,level,reps,arch,demog_name,demog_level_sub,demog_level,MIG_x,MIG_y,xcline,ycline,demog,METAPOP_SIDE_x,METAPOP_SIDE_y,Nequal,isVariableM,MIG_breaks,arch_level_sub,arch_level,MU_base,MU_QTL_proportion,SIGMA_QTN_1,SIGMA_QTN_2,SIGMA_K_1,SIGMA_K_2,N_traits,ispleiotropy,seed
1231094,highly-polygenic_1-trait__Est-Clines_N-cline-c...,1,highly-polygenic_1-trait,Est-Clines_N-cline-center-to-edge_m-constant,N-cline-center-to-edge_m-constant,Est-Clines,0.49,0.07,linear,linear,Estuary,10,10,4,0,0,1-trait,highly-polygenic,1.000000e-07,0.25,0.002,0.002,0.5,0.5,1,0,1231094
1231095,highly-polygenic_1-trait__Est-Clines_N-cline-N...,1,highly-polygenic_1-trait,Est-Clines_N-cline-N-to-S_m-constant,N-cline-N-to-S_m-constant,Est-Clines,0.49,0.07,linear,linear,Estuary,10,10,2,0,0,1-trait,highly-polygenic,1.000000e-07,0.25,0.002,0.002,0.5,0.5,1,0,1231095


In [9]:
jobs = []
args = []
for seed in pbar(seeds):
    for climate_val, (opt0, opt1) in envdata.items():
        outfile = op.join(fitness_dir, f'{seed}_{climate_val}.txt')
        
        n_traits = params.loc[seed, 'N_traits']
        
        if n_traits == 2:
            arg = (seed, outfile, opt1, opt0)
        
        else:
            arg = (seed, outfile, opt1)
            
        args.append(arg)
        
        jobs.append(
            lview.apply_async(
                calc_R_fitness, *(arg)
            )
        )
        
watch_async(jobs)

100%|███████████████| 2250/2250 [00:51<00:00, 43.45it/s]



Watching 27000 jobs ...


100%|███████████████| 27000/27000 [18:01<00:00, 24.96it/s] 


In [10]:
# make sure no errors
for j in jobs:
    x = j.r  # returns None if no error